In [1]:
import glob
import os
import subprocess

def convert(n, d):
    get_ipython().system("jupyter nbconvert {} --to markdown --output {}".format(n, d))
    #subprocess.check_call(["jupyter", "nbconvert", n, "--to", "markdown", "--output", d])

highlevel_dirs = ["../tools"] + sorted(glob.glob("../sem*"))

print("Highlevel dirs:", highlevel_dirs)

Highlevel dirs: ['../tools', '../sem01', '../sem03-ints-floats', '../sem04-asm-arm', '../sem05-asm-arm-addressing', '../sem06-asm-x86', '../sem07-asm-x86-x87-sse', '../sem08-asm-x86-nostdlib', '../sem09-low-level-io', '../sem10-file-attributes', '../sem11-mmap-instrumentation', '../sem12-fork-exec-pipe', '../sem13-signal', '../sem14-fifo-proc', '../sem15-ptrace', '../sem16-fcntl-dup-pipe']


In [2]:
for subdir in highlevel_dirs:
    notebooks = glob.glob(subdir + "/*.ipynb")
    print(subdir, notebooks)
    for m in glob.glob(subdir + "/*.md"):
        os.remove(m)
    if len(notebooks) == 1:
        convert(notebooks[0], "README")
    else:
        for n in notebooks:
            convert(n, os.path.basename(n.replace(".ipynb", "")))
        

../tools ['../tools/set_up_magics.ipynb', '../tools/set_up_magics_dev.ipynb', '../tools/save_them_all.ipynb']
[NbConvertApp] Converting notebook ../tools/set_up_magics.ipynb to markdown
[NbConvertApp] Writing 50748 bytes to ../tools/set_up_magics.md
[NbConvertApp] Converting notebook ../tools/set_up_magics_dev.ipynb to markdown
[NbConvertApp] Writing 32 bytes to ../tools/set_up_magics_dev.md
[NbConvertApp] Converting notebook ../tools/save_them_all.ipynb to markdown
[NbConvertApp] Writing 20343 bytes to ../tools/save_them_all.md
../sem01 ['../sem01/sem1.ipynb']
[NbConvertApp] Converting notebook ../sem01/sem1.ipynb to markdown
[NbConvertApp] Writing 2135 bytes to ../sem01/README.md
../sem03-ints-floats ['../sem03-ints-floats/sem3_floats.ipynb', '../sem03-ints-floats/sem3_ints.ipynb']
[NbConvertApp] Converting notebook ../sem03-ints-floats/sem3_floats.ipynb to markdown
[NbConvertApp] Writing 2857 bytes to ../sem03-ints-floats/sem3_floats.md
[NbConvertApp] Converting notebook ../sem03-in

In [3]:
import re

def basic_improve(fname):
    with open(fname, "r") as f:
        r = f.read()
    for b in ["\x00", "\x1B", "\x08"]:
        r = r.replace(b, "")
    with open(fname, "w") as f:
        f.write(r)
    get_ipython().system("dos2unix {}".format(fname))

def improve_md(fname):
    with open(fname, "r") as f:
        r = f.read()
    r = r.replace("```python\n%%cpp", "```cpp\n%%cpp")
    r = r.replace('\n', "SUPER_SLASH" + "_N_REPLACER")
    r = re.sub(r'\<\!--MD_BEGIN_FILTER--\>.*?\<\!--MD_END_FILTER--\>', "", r)
    r = re.sub(r'(\#SET_UP_MAGIC_BEGIN.*?\#SET_UP_MAGIC_END)', "<too much code>", r)
    r = r.replace("SUPER_SLASH" + "_N_REPLACER", '\n')
    
    def file_repl(matchobj, path=os.path.dirname(fname)):
        fname = os.path.join(path, matchobj.group(1))
        if fname.find("__FILE__") == -1:
            with open(fname, "r") as f:
                return "\n```\n" + f.read() + "\n```\n"
    
    r = r.replace("</td>", "")
    r = r.replace("</tr>", "")
    
    r = re.sub(r'\<\!--MD_FROM_FILE (.*?) --\>', file_repl, r)
    with open(fname, "w") as f:
        f.write(r)
        
def improve_file(fname):
    basic_improve(fname)
    if fname.endswith(".md"):
        improve_md(fname)


In [4]:
for sfx in [".ipynb", ".md"]:
    for hdir in highlevel_dirs:
        for fname in glob.glob("./{}/*".format(hdir) + sfx):
            improve_file(fname)

dos2unix: converting file ./../tools/set_up_magics.ipynb to Unix format ...
dos2unix: converting file ./../tools/set_up_magics_dev.ipynb to Unix format ...
dos2unix: converting file ./../tools/save_them_all.ipynb to Unix format ...
dos2unix: converting file ./../sem01/sem1.ipynb to Unix format ...
dos2unix: converting file ./../sem03-ints-floats/sem3_floats.ipynb to Unix format ...
dos2unix: converting file ./../sem03-ints-floats/sem3_ints.ipynb to Unix format ...
dos2unix: converting file ./../sem04-asm-arm/arm.ipynb to Unix format ...
dos2unix: converting file ./../sem05-asm-arm-addressing/adressing.ipynb to Unix format ...
dos2unix: converting file ./../sem06-asm-x86/asm_x86.ipynb to Unix format ...
dos2unix: converting file ./../sem07-asm-x86-x87-sse/sse_x86.ipynb to Unix format ...
dos2unix: converting file ./../sem07-asm-x86-x87-sse/floating_point_x86.ipynb to Unix format ...
dos2unix: converting file ./../sem08-asm-x86-nostdlib/nostdlib.ipynb to Unix format ...
dos2unix: convert

In [13]:
add_cmd = "git add --ignore-errors "
add_cmd_f = "git add"
for subdir in highlevel_dirs:
    for sfx in [".ipynb", ".md", ".c", ".cpp"]:
        add_cmd += " {}/*{}".format(subdir, sfx)
    add_cmd_f += " -f {}/bash_popen_tmp/*.html".format(subdir)
    
def execute_cmd(cmd):
    print(">", cmd)
    get_ipython().system(cmd)
    
execute_cmd(add_cmd)
execute_cmd(add_cmd_f)
execute_cmd("git add -u")
execute_cmd("git commit -m 'yet another update'")
execute_cmd("git push origin master")

> git add --ignore-errors  ../tools/*.ipynb ../tools/*.md ../tools/*.c ../tools/*.cpp ../sem01/*.ipynb ../sem01/*.md ../sem01/*.c ../sem01/*.cpp ../sem03-ints-floats/*.ipynb ../sem03-ints-floats/*.md ../sem03-ints-floats/*.c ../sem03-ints-floats/*.cpp ../sem04-asm-arm/*.ipynb ../sem04-asm-arm/*.md ../sem04-asm-arm/*.c ../sem04-asm-arm/*.cpp ../sem05-asm-arm-addressing/*.ipynb ../sem05-asm-arm-addressing/*.md ../sem05-asm-arm-addressing/*.c ../sem05-asm-arm-addressing/*.cpp ../sem06-asm-x86/*.ipynb ../sem06-asm-x86/*.md ../sem06-asm-x86/*.c ../sem06-asm-x86/*.cpp ../sem07-asm-x86-x87-sse/*.ipynb ../sem07-asm-x86-x87-sse/*.md ../sem07-asm-x86-x87-sse/*.c ../sem07-asm-x86-x87-sse/*.cpp ../sem08-asm-x86-nostdlib/*.ipynb ../sem08-asm-x86-nostdlib/*.md ../sem08-asm-x86-nostdlib/*.c ../sem08-asm-x86-nostdlib/*.cpp ../sem09-low-level-io/*.ipynb ../sem09-low-level-io/*.md ../sem09-low-level-io/*.c ../sem09-low-level-io/*.cpp ../sem10-file-attributes/*.ipynb ../sem10-file-attributes/*.md ../sem1